In [ ]:
from baseline import SRCNN
from baseline import FSRCNN
from train_utils import validation
from datasets.data import DIV2K

from argparse import Namespace
import torch.optim as optim
import torch.nn as nn
import torch

import warnings
warnings.filterwarnings("ignore")

model = SRCNN(True)

state_dict = torch.load("D:\\Models\SuperRes\\srcnn_x4.pth")
adapted_state_dict = {}
for k, v in state_dict.items():
    if k.startswith('conv1.'):
        adapted_k = k.replace('conv1.', 'features.0.')
    elif k.startswith('conv2.'):
        adapted_k = k.replace('conv2.', 'map.0.')
    elif k.startswith('conv3.'):
        adapted_k = k.replace('conv3.', 'reconstruction.')
    else:
        adapted_k = k
        
    adapted_state_dict[adapted_k] = v

model.load_state_dict(adapted_state_dict)

model.to('cuda')

valid_dataset = DIV2K('D:\\DATA\\DIV2K', train=False, gray_scale=True)

valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=32)

loss, psnr, ssim = validation(model, valid_loader, nn.MSELoss(), 'cuda', 0)

print(loss, psnr.item() / len(valid_loader), ssim.item() / len(valid_loader))